# 1 Setup.

In [ ]:
### Imports. ###
import matplotlib.pyplot as plt  # 3.10.0.
import matplotlib.ticker as ticker # 3.10.0.
import numpy as np  # 2.1.3.
import pandas as pd  # 2.2.3.
import re 
import seaborn as sns  # 0.13.2.
import statsmodels.api as sm  # 0.14.4.

from io import StringIO  # 3.13.5.
from linearmodels.panel import PanelOLS  # 6.1.
from matplotlib.lines import Line2D # 3.10.0.

# Enable LaTeX rendering in plots.
plt.rcParams['text.usetex'] = True

# Make output folders.
import os
os.makedirs("tabs", exist_ok=True)
os.makedirs("figs", exist_ok=True)

In [ ]:
### Global variables. ###
dependent_vars_raw = ['CON_t+1', 'CON_t+2', 'CON_t+3', 'CON_t+4', 'CON_t+5', 'CON_t^3', 'CON_t^5']
dependent_vars = [f'log({var})' for var in dependent_vars_raw]

model_specs = [['log(STR)'],
               ['log(STR)','log(POP)'],
               ['log(STR)','log(POP)','log(SGDP)'],
               ['log(STR)','log(POP)','log(SGDP)','log(POL)']]

In [ ]:
### Helper functions. ###

def human_format(x, pos=None):
    """
    Format tick values into K/M format.
    """
    if abs(x) >= 1e6:
        return f'{x/1e6:.1f}M'
    elif abs(x) >= 1e3:
        return f'{x/1e3:.0f}K'
    else:
        return f'{x:.0f}'

def format_with_commas(df):
    r"""
    Format numeric DataFrame cells with commas and 2 decimal places.
    """
    return df.map(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)

def add_phantom_to_latex_table(latex_table):
    r"""
    Add \phantom{-} before positive numbers (for LaTeX alignment).
    Assumes numbers are formatted with two decimal places.
    """
    return re.sub(r'(?<![-\d.])(\d+\.\d\d)', r'\\phantom{-}\1', latex_table)

def generate_latex_table_with_results(df, params):
    r"""
    Generate a LaTeX-formatted table from a DataFrame of model results.
    Results are grouped by dependent variable.
    """
    df_clean = df.copy()
    base_cols = ['Model ID', '$n$', '$R^{2}$'] 
    header_cols = base_cols + params
    output = StringIO()

    # Begin tabular.
    output.write(f"\\begin{{tabular}}{{{'c' + 'c' * (len(header_cols) - 1)}}}\n")
    output.write("\\toprule\n")
    output.write(" & ".join(header_cols) + " \\\\\n")
    output.write("\\midrule\\midrule\n")

    # Iterate over each dependent variable.
    for idx, dep_var in enumerate(dependent_vars):
        dep_label = f"${dep_var}$"
        if idx > 0:
            output.write("\\midrule\n")
        output.write(f"\\multicolumn{{{len(header_cols)}}}{{l}}{{Dependent variable: {dep_label}}} \\\\\n")
        output.write("\\midrule\n")

        block = df_clean[df_clean['Dependent variable'] == dep_var]
        for _, row in block.iterrows():
            row_cells = ["" if pd.isna(row[col]) else str(row[col]) for col in header_cols]
            output.write(" & ".join(row_cells) + " \\\\\n")

    output.write("\\bottomrule\n\\end{tabular}")
    latex_table = output.getvalue()

    # Static replacements.
    replacements = {'nan': '',
                    'const': '$\\alpha$',
                    'log(STR)': r'$\log_{10}\left(STR_{(c,t)}\right)$',
                    'log(POP)': r'$\log_{10}\left(POP_{(c,t)}\right)$',
                    'log(SGDP)': r'$\log_{10}\left(SGDP_{(c,t)}\right)$',
                    'log(POL)': r'$\log_{10}\left(POL_{(c,t)}\right)$',
                    '$log(CON_t^3)$': r'$\log_{10}(CON_{\left(c,t+1:3\right)})$',
                    '$log(CON_t^5)$': r'$\log_{10}(CON_{\left(c,t+1:5\right)})$',
                    '& t ': r'& $ t $'}

    for old, new in replacements.items():
        latex_table = latex_table.replace(old, new)

    # Dynamic replacements for log(CON_t+N).
    for j in range(6):
        search = f'$log(CON_t+{j})$'
        replace = f'$\\log_{{10}}\\left(CON_{{(c,t+{j})}}\\right)$'
        latex_table = latex_table.replace(search, replace)

    # Replace time labels t_0, t_1, ..., t_8.
    for i in range(9):
        latex_table = latex_table.replace(f't_{i}', f'$t={i}$')

    return latex_table

In [ ]:
### Load and prepare data. ###
df_excel = pd.ExcelFile('dataset.xlsx')

# Parse and reshape all sheets, then merge into one DataFrame.
df = pd.concat([df_excel.parse(sheet).melt(id_vars="YEAR", var_name="COUNTRY", value_name=sheet)
    for sheet in df_excel.sheet_names]).pivot_table(index=["YEAR", "COUNTRY"]).reset_index()

### Create derived variables. ###
df['SGDP'] = (df['SHW'] / 100) * df['GDP'] # SHW is a percentage
df['t'] = df['YEAR'] - 2006 # Time since baseline year

### Reorder and sort data. ###
df = df[['COUNTRY', 'YEAR', 'STR', 'GDP', 'SHW', 'SGDP', 'POL', 'CON', 'POP', 't']]
df = df.sort_values(by=['COUNTRY', 'YEAR'], ascending=False)

### Create lagged CON variables (t+1 to t+5). ###
for j in range(1, 6):
    df[f'CON_t+{j}'] = df.groupby('COUNTRY')['CON'].transform(lambda x: x.shift(j))

### Create rolling averages of lagged CON (3- and 5-year). ###
for H in [3, 5]:
    df[f'CON_t^{H}'] = df.groupby('COUNTRY')['CON'].transform(lambda x: x.shift(1).rolling(H).mean())

### Measure per 100,000 capita and apply log10 transformation. ###
for col in ['STR', 'GDP','SGDP','POL','CON'] + dependent_vars_raw:
    if col in ['GDP','SGDP']:
        df[col] = (df[col] / df['POP'])
    else:
        df[col] = (df[col] / df['POP'])*1e5

    # Log10 transform (only for positive values).
    log_col = pd.Series(-np.inf, index=df.index)
    mask = df[col] > 0
    log_col[mask] = np.log10(df.loc[mask, col])
    df[f'log({col})'] = log_col

# Log10 transform POP.
df['log(POP)'] = np.log10(df['POP'])

# 2 Data understanding and visualization.

In [ ]:
### View data. ###
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
df

In [ ]:
### Count and show zero observations of CON. ###
print(f'Number of CON observations with zero: {len(df[df["CON"] == 0])}')
print(df[df["CON_t^3"] == 0])
print(df[df["CON_t^5"] == 0])

## 2.1 Calculate statistics and correlations.

In [ ]:
### Summary Statistics and LaTeX Table Export. ###
# Select columns for summary statistics.
cols = ['STR', 'GDP', 'SHW', 'SGDP', 'POL', 'CON', 'POP', 't']

# Compute basic statistics.
stats = df[cols].agg(['mean', 'std', 'min', 'median', 'max'])
stats.index = ['Mean', 'Std. dev.', 'Minimum', 'Median', 'Maximum']

# Convert to LaTeX format.
latex_table = format_with_commas(stats).to_latex(column_format='r' + 'r' * len(stats.columns))

# Replace header row with custom LaTeX-formatted variable names.
latex_table = latex_table.replace(' & STR & GDP & SHW & SGDP & POL & CON & POP & t',
r"""\multicolumn{1}{c}{} & \multicolumn{1}{c}{$STR_{(c,t)}$} & \multicolumn{1}{c}{$GDP_{(c,t)}$} & 
\multicolumn{1}{c}{$SHW_{(c,t)}$} & \multicolumn{1}{c}{$SGDP_{(c,t)}$} &
\multicolumn{1}{c}{$POL_{(c,t)}$} & \multicolumn{1}{c}{$CON_{(c,t)}$} & 
\multicolumn{1}{c}{$POP_{(c,t)}$} & \multicolumn{1}{c}{$t$}""")

# Write LaTeX table to file
with open('tabs/stats.txt', 'w') as file:
    file.write(latex_table)

# View stats.
stats

In [ ]:
### Correlation Matrix and LaTeX Table Export. ###
# Select relevant columns.
cols = ['STR', 'GDP', 'SHW', 'SGDP', 'POL', 'CON', 'POP', 't']

# Calculate correlation matrix.
corr = df[cols].corr()

# Convert to LaTeX format.
latex_table = format_with_commas(corr).to_latex(column_format='c' + 'c' * len(cols))

# Replace LaTeX header row with formatted variable names.
latex_table = latex_table.replace(' & STR & GDP & SHW & SGDP & POL & CON & POP & t',
r"""\multicolumn{1}{c}{} & \multicolumn{1}{c}{$STR_{(c,t)}$} & \multicolumn{1}{c}{$GDP_{(c,t)}$} & 
\multicolumn{1}{c}{$SHW_{(c,t)}$} & \multicolumn{1}{c}{$SGDP_{(c,t)}$} &
\multicolumn{1}{c}{$POL_{(c,t)}$} & \multicolumn{1}{c}{$CON_{(c,t)}$} & 
\multicolumn{1}{c}{$POP_{(c,t)}$} & \multicolumn{1}{c}{$t$}""")

# Replace row labels with LaTeX math equivalents, in reverse to avoid name conflicts.
for col in reversed(cols):
    if col != 't':
        latex_table = latex_table.replace(f'{col} ', f'${col}_{{(c,t)}}$ ')

# Add \phantom{-} to positive values for alignment in LaTeX.
latex_table = add_phantom_to_latex_table(latex_table)

# Save LaTeX table to file.
with open('tabs/corr.txt', 'w') as latex_file:
    latex_file.write(latex_table)

# View correlations.
corr

## 2.1 Plot of 'CON' by country.

In [ ]:
### Count Observations of CON by Country. ###
country_obs = df.groupby('COUNTRY')['CON'].count().reset_index()
country_obs.columns = ['COUNTRY', 'CON Count']

### Plot Bar Chart. ###
plt.figure(figsize=(8, 3))

# Plot using seaborn's deep color palette
sns.barplot(data=country_obs, x='COUNTRY', y='CON Count', color=sns.color_palette("deep")[0])

# Axis adjustments.
ax = plt.gca()
ax.set_xlim(-1, len(country_obs['COUNTRY']))  # Add horizontal space.
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))  # Gridline every 1 unit.

# Grid and labels.
plt.grid(True, linestyle='--', alpha=0.3)
plt.xlabel('')
plt.ylabel(r'Observations of $CON_{(c,t)}$', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Show and export.
plt.tight_layout()
plt.savefig('figs/country.pdf')
plt.show()

## 2.2 Plot of 'CON' by year.

In [ ]:
### Count Observations of CON by Year. ###
year_obs = df.groupby('YEAR')['CON'].count().reset_index()
year_obs.columns = ['YEAR', 'CON Count']

# Compute t-values relative to the first year.
t_values = year_obs['YEAR'] - year_obs['YEAR'].min()

# Create LaTeX-formatted xtick labels.
xtick_labels = [fr"$t={t}$ ({year})" for t, year in zip(t_values, year_obs['YEAR'])]

### Plot Line Chart. ###
plt.figure(figsize=(8, 4))

# Line plot with markers.
plt.plot(year_obs['YEAR'], year_obs['CON Count'], marker='o', linestyle='-', linewidth=2)

# X and Y ticks.
plt.xticks(ticks=year_obs['YEAR'], labels=xtick_labels, fontsize=10, rotation=45)
plt.yticks(ticks=range(0, year_obs['CON Count'].max() + 3), fontsize=10)

# Grid and labels.
plt.grid(True, linestyle='--', alpha=0.3)
plt.xlabel('')
plt.ylabel(r'Observations of $CON_{(c,t)}$', fontsize=12)

# Show and export.
plt.tight_layout()
plt.savefig('figs/year.pdf')
plt.show()

## 2.3 Average percentage increase in CON and STRs between 2006 and 2014

In [ ]:
# Filter for years 2006 and 2014 and drop rows with missing STR values
df_filtered = df[df['YEAR'].isin([2006, 2014])][['COUNTRY', 'YEAR', 'STR']].dropna()

# Pivot the data to have years as columns
df_pivot = df_filtered.pivot(index='COUNTRY', columns='YEAR', values='STR')

# Calculate development: absolute and percentage change from 2006 to 2014
df_pivot['Absolute Change'] = df_pivot[2014] - df_pivot[2006]
df_pivot['Percentage Change'] = ((df_pivot[2014] - df_pivot[2006]) / df_pivot[2006]) * 100

average_percentage_STR_change = df_pivot['Percentage Change'].mean()
print(f'Average percentage increase in STRs between 2006 and 2014: {average_percentage_STR_change:.2f}%')

median_percentage_STR_change = df_pivot['Percentage Change'].median()
print(f'Median percentage increase in STRs between 2006 and 2014: {median_percentage_STR_change:.2f}%')

df_pivot['Percentage Change']

# 3 Pooled regression.

In [ ]:
### Pooled OLS Regressions with clustered standard errors. ###
results = pd.DataFrame()
for dep_var in dependent_vars:
    for i, exog_vars in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + exog_vars)
        X, y = sm.add_constant(temp_df[exog_vars]), temp_df[dep_var]
        res = sm.OLS(y, X).fit(cov_type="cluster",cov_kwds={"groups": temp_df["COUNTRY"].to_numpy()})
        print(res.summary())

        # Extract results.
        row = {'Dependent variable': dep_var, 'Model ID': f'P{i}', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared:.2f}"}
        for p in res.params.index:
            coef, se = res.params[p], res.bse[p]
            stars = ('***' if res.pvalues[p] < 0.001 else
                     '**' if res.pvalues[p] < 0.01 else
                     '*' if res.pvalues[p] < 0.05 else '')
            formatted = f"{coef:.2f}{stars} ({se:.2f})"
            row[p] = formatted
        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export results.
latex_table = generate_latex_table_with_results(results, params=['const','log(STR)','log(POP)','log(SGDP)','log(POL)'])
with open('tabs/pooled_res.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results

## 3.1 Plot.

In [ ]:
### Pooled Regression Plot: log(CON_t^3) ~ log(STR) (Model P1). ###

# Prepare regression data and run model.
temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['log(CON_t^3)', 'log(STR)']).copy()
temp_df['YEAR'] = '$t=' + (temp_df['YEAR'] - 2006).astype(str) + '$ (' + temp_df['YEAR'].astype(str) + ')' # Format years.
X, y = sm.add_constant(temp_df['log(STR)']), temp_df['log(CON_t^3)']

res = sm.OLS(y, X).fit(cov_type="cluster",cov_kwds={"groups": temp_df["COUNTRY"].to_numpy()})
print(res.summary())

temp_df['fitted_values'] = res.predict(X)

# Setup plotting.
marker_styles = ['s', 'v', '^', '<', '>', 'P', 'p', '*', 'D', 'd', 'X']
countries, years = temp_df['COUNTRY'].unique(), temp_df['YEAR'].unique()
country_markers = {c: marker_styles[i % len(marker_styles)] for i, c in enumerate(countries)}
year_colors = dict(zip(years, sns.color_palette('tab10', len(years))))
legend_marker_styles, marker_usage = {}, {m: 0 for m in marker_styles}
plt.figure(figsize=(14, 8))

# Scatter plot by country-year.
for country in countries:
    for _, row in temp_df[temp_df['COUNTRY'] == country].iterrows():
        marker, year = country_markers[country], row['YEAR']
        facecolor = year_colors[year] if marker_usage[marker] == 0 else 'none'
        plt.scatter(row['log(STR)'], row['log(CON_t^3)'], s=150, marker=marker,
                    facecolor=facecolor, edgecolor=year_colors[year], linewidth=2)
    legend_marker_styles[country] = {'marker': marker, 'facecolor': 'black' if marker_usage[marker] == 0 else 'none'}
    marker_usage[marker] += 1

# Add fitted regression line.
sns.lineplot(x=temp_df['log(STR)'], y=temp_df['fitted_values'], color='black', label='Fitted Line', linestyle='--', lw=2)

# Legend for years (color).
legend1 = plt.legend(handles=[Line2D([0], [0], color=c, marker='o', linestyle='', label=str(y)) for y, c in year_colors.items()],
                     loc='upper left', bbox_to_anchor=(1.01, 1), title="Year", frameon=True)
plt.gca().add_artist(legend1)

# Legend for countries (marker).
legend2 = plt.legend(
    handles=[Line2D([0], [0], marker=info['marker'], linestyle='', markerfacecolor=info['facecolor'],
                    markeredgecolor='black', markersize=10, label=country)
             for country, info in legend_marker_styles.items()],
    loc='upper left', bbox_to_anchor=(1.01, 0.672), title='Country', frameon=True)

# Format legends.
for legend in [legend1, legend2]:
    legend.get_frame().set_boxstyle("square,pad=0.4")

# Axis labels with LaTeX.
plt.xlabel(r'$\log_{10}\left(STR_{(c,t)}\right)$', fontsize=14)
plt.ylabel(r'$\log_{10}(CON_{\left(c,t+1:3\right)})$', fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

# Show and export.
plt.tight_layout()
plt.savefig('figs/pooled_reg.pdf')
plt.show()

## 3.2 Country robustness

In [ ]:
### Sequential Exclusion of Countries in Pooled Regressions. ###
countries = df['COUNTRY'].unique()
results = pd.DataFrame()

for country in countries:
    df_excluded = df[df['COUNTRY'] != country]

    for dep_var in dependent_vars:
        for i, exog_vars in enumerate(model_specs, start=1):
            
            # Run model.
            temp_df = df_excluded.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + exog_vars)
            X, y = sm.add_constant(temp_df[exog_vars]), temp_df[dep_var]
            res = sm.OLS(y, X).fit(cov_type="cluster",cov_kwds={"groups": temp_df["COUNTRY"].to_numpy()})

            # Extract result.
            row = {'Excluded country': country, 'Dependent variable': dep_var, 'Model ID': f'P{i}', '$n$': int(res.nobs), '$R^{2}$': res.rsquared}
            for p in res.params.index:
                row[f'{p} coef'], row[f'{p} se'], row[f'{p} p-value'] = res.params[p], res.bse[p], res.pvalues[p]

            results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Filter to show where log(STR) is insignificant.
results[results['log(STR) p-value'] >= 0.05]

## 3.3 Time robustness.

In [ ]:
### Pooled OLS with Log-Linear Time Trend. ###
results = pd.DataFrame()
for dep_var in dependent_vars:
    for i, base_exog in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + base_exog + ['t'])
        X = sm.add_constant(temp_df[base_exog + ['t']])
        y = temp_df[dep_var]
        res = sm.OLS(y, X).fit(cov_type="cluster",cov_kwds={"groups": temp_df["COUNTRY"].to_numpy()})
        print(res.summary())

        # Extract result.
        row = {'Dependent variable': dep_var, 'Model ID': f'P{i}T', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared:.2f}"}
        for p in res.params.index:
            coef, se = res.params[p], res.bse[p]
            stars = ('***' if res.pvalues[p] < 0.001 else '**' if res.pvalues[p] < 0.01 else '*' if res.pvalues[p] < 0.05 else '')
            row[p] = f"{coef:.2f}{stars} ({se:.2f})"
        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export results.
latex_table = generate_latex_table_with_results(results, params=['const', 'log(STR)','log(POP)','log(SGDP)','log(POL)','t'])
with open('tabs/pooled_time_trend.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results

In [ ]:
### Pooled OLS with Year Dummies. ###
results = pd.DataFrame()
for dep_var in dependent_vars:
    for i, base_exog in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + base_exog + ['t'])
        year_dummies = pd.get_dummies(temp_df['t'], prefix='t', drop_first=True).astype(int)
        X = sm.add_constant(pd.concat([temp_df[base_exog], year_dummies], axis=1))
        y = temp_df[dep_var]
        res = sm.OLS(y, X).fit(cov_type="cluster",cov_kwds={"groups": temp_df["COUNTRY"].to_numpy()})
        print(res.summary())

        # Extract result.
        row = {'Dependent variable': dep_var, 'Model ID': f'P{i}TD', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared:.2f}"}
        for p in res.params.index:
            coef, se = res.params[p], res.bse[p]
            stars = ('***' if res.pvalues[p] < 0.001 else '**' if res.pvalues[p] < 0.01 else '*' if res.pvalues[p] < 0.05 else '')
            row[p] = f"{coef:.2f}{stars} ({se:.2f})"
        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export results.
t_vars = results.filter(regex=r'^t_\d+$').columns.tolist()
latex_table = generate_latex_table_with_results(results, params=['const','log(STR)','log(POP)','log(SGDP)','log(POL)'] + t_vars)
with open('tabs/pooled_time_dummies.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results

# 4 Fixed effects regression.

In [ ]:
### Fixed Effects (PanelOLS) Regressions with Clustered SEs. ###
results = pd.DataFrame()

for dep_var in dependent_vars:
    for i, exog_vars in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + exog_vars).copy()
        temp_df = temp_df.set_index(['COUNTRY', 'YEAR'])
        y, X = temp_df[dep_var], temp_df[exog_vars]
        model = PanelOLS(y, X, entity_effects=True)
        res = model.fit(cov_type='clustered', cluster_entity=True)
        print(res.summary)

        # Extract results.
        row = {'Dependent variable': dep_var,
               'Model ID': f'F{i}',
               '$n$': f'{int(res.nobs)}',
               '$R^{2}$': f'{res.rsquared_within:.2f}'}
            
        # Coefficient + stars + standard error.
        for p in res.params.index:
            coef, se = res.params[p], res.std_errors[p]
            stars = ('***' if res.pvalues[p] < 0.001 else '**' if res.pvalues[p] < 0.01 else '*' if res.pvalues[p] < 0.05 else '')
            row[p] = f"{coef:.2f}{stars} ({se:.2f})"

        stars = ('***' if res.f_pooled.pval < 0.001 else '**' if res.f_pooled.pval < 0.01 else '*' if res.f_pooled.pval < 0.05 else '')
        row['F (poolability)'] = f'{res.f_pooled.stat:.2f}{stars}'

        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export to LaTeX.
latex_table = generate_latex_table_with_results(results, params=['F (poolability)','log(STR)','log(POP)','log(SGDP)','log(POL)'])
latex_table = latex_table.replace('$R^{2}$', '$R^{2}$ (within)')

with open('tabs/fixed_res.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results


## 4.1 Plot fixed effect regression

In [ ]:
### Fit Fixed Effects Model: log(CON_t^3) ~ log(STR) (Model F1). ###

# Drop missing values and set panel index.
temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['log(CON_t^3)', 'log(STR)']).copy()
temp_df = temp_df.set_index(['COUNTRY', 'YEAR'])

# Fit model and extract predictions.
model = PanelOLS(temp_df['log(CON_t^3)'], temp_df[['log(STR)']], entity_effects=True)
res = model.fit(cov_type='clustered', cluster_entity=True)
print(res.summary)

# Add predictions and reset index.
temp_df['predictions'] = res.predict(temp_df[['log(STR)']]).values.flatten() + res.estimated_effects.values.flatten()
temp_df = temp_df.reset_index()

# Estimate country fixed effects.
fixed_effects = res.estimated_effects.reset_index().groupby('COUNTRY')['estimated_effects'].mean().reset_index()
fixed_effects.columns = ['Country', '$\\alpha_{(c)}$']
fixed_effects['$A_{(c)}$'] = 10 ** fixed_effects['$\\alpha_{(c)}$']
fixed_effects = fixed_effects.sort_values(by='$\\alpha_{(c)}$', ascending=False).reset_index(drop=True)

# Export fixed effects to LaTeX.
latex = format_with_commas(fixed_effects).to_latex(index=False).replace('lll', 'lrr')
with open('tabs/alphas.txt', 'w') as f:
    f.write(latex)

### Plot Observed and Predicted Values. ###
marker_styles = ['s', 'v', '^', '<', '>', 'P', 'p', '*', 'D', 'd', 'X']
temp_df['YEAR'] = '$t=' + (temp_df['YEAR'] - 2006).astype(str) + '$ (' + temp_df['YEAR'].astype(str) + ')' # Format years.
countries, years = temp_df['COUNTRY'].unique(), temp_df['YEAR'].unique()
country_markers = {c: marker_styles[i % len(marker_styles)] for i, c in enumerate(countries)}
year_colors = dict(zip(years, sns.color_palette('tab10', len(years))))
legend_marker_styles, marker_usage = {}, {m: 0 for m in marker_styles}

plt.figure(figsize=(14, 8))

for country in countries:
    marker = country_markers[country]
    country_data = temp_df[temp_df['COUNTRY'] == country]
    for _, row in country_data.iterrows():
        facecolor = year_colors[row['YEAR']] if marker_usage[marker] == 0 else 'none'
        plt.scatter(row['log(STR)'], row['log(CON_t^3)'], s=150, marker=marker,
                    facecolor=facecolor, edgecolor=year_colors[row['YEAR']], linewidth=2)

    legend_marker_styles[country] = {'marker': marker, 'facecolor': 'black' if marker_usage[marker] == 0 else 'none'}

    # Plot fitted line per country.
    sorted_data = country_data.sort_values(by='log(STR)')
    sns.lineplot(x=sorted_data['log(STR)'], y=sorted_data['predictions'], color='black',
                 linestyle='--', lw=2, legend=False)

    # Mark endpoints.
    for idx in [0, -1]:
        plt.scatter(sorted_data['log(STR)'].iloc[idx], sorted_data['predictions'].iloc[idx],
                    s=50, marker=marker, facecolor='black' if marker_usage[marker] == 0 else 'none',
                    edgecolor='black', linewidth=1)

    marker_usage[marker] += 1

# Year color legend.
legend1 = plt.legend(handles=[Line2D([0], [0], color=c, marker='o', linestyle='', label=str(y)) for y, c in year_colors.items()], 
                     loc='upper left', bbox_to_anchor=(1.01, 1), title="Year", frameon=True)
plt.gca().add_artist(legend1)

# Country marker legend.
legend2 = plt.legend(handles=[
    Line2D([0], [0], marker=info['marker'], linestyle='', markerfacecolor=info['facecolor'],
           markeredgecolor='black', markersize=10, label=country)
    for country, info in legend_marker_styles.items()], loc='upper left', bbox_to_anchor=(1.01, 0.672), title="Country (Symbol)", frameon=True)

for legend in [legend1, legend2]:
    legend.get_frame().set_boxstyle("square,pad=0.4")

# Axis labels and styling.
plt.xlabel(r'$\log_{10}\left(STR_{(c,t)}\right)$', fontsize=14)
plt.ylabel(r'$\log_{10}(CON_{\left(c,t+1:3\right)})$', fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

# Show and export.
plt.tight_layout()
plt.savefig('figs/fixed_reg.pdf')
plt.show()

In [ ]:
### Show fixed-effects results. ### 
fixed_effects

## 4.2 Country robustness analysis

In [ ]:
### Sequential Fixed Effects Regressions (Exclude One Country; Model F1). ###
countries = df['COUNTRY'].unique()
results = pd.DataFrame()
            
for country in countries:
    df_excluded = df[df['COUNTRY'] != country]

    for dep_var in dependent_vars:
        for i, exog_vars in enumerate(model_specs, start=1):

            # Run model.
            temp_df = df_excluded.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var]+exog_vars).copy()
            temp_df = temp_df.set_index(['COUNTRY', 'YEAR'])
            y, X = temp_df[dep_var], temp_df[exog_vars]
            model = PanelOLS(y, X, entity_effects=True)
            res = model.fit(cov_type='clustered', cluster_entity=True)
    
            # Extract result.
            row = {'Excluded country': country, 'Dependent variable': dep_var, 'Model ID': f'P{i}', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared_within:.2f}"}
            for p in res.params.index:
                row[f'{p} coef'], row[f'{p} se'], row[f'{p} p-value'] = res.params[p], res.std_errors[p], res.pvalues[p]
    
            stars = ('***' if res.f_pooled.pval < 0.001 else '**' if res.f_pooled.pval < 0.01 else '*' if res.f_pooled.pval < 0.05 else '')
            row['F (poolability)'] = f'{res.f_pooled.stat:.2f}{stars}'
    
            results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

In [ ]:
# Inspect excluded countries that make log(STR) insignificant for log(CON_t+1), log(CON_t+2), log(CON_t+3), or log(CON_t^3).
results[(results['log(STR) p-value'] >= 0.05) 
        & ((results['Dependent variable'] == 'log(CON_t+1)')
           | (results['Dependent variable'] == 'log(CON_t+2)')
           | (results['Dependent variable'] == 'log(CON_t+3)')
           | (results['Dependent variable'] == 'log(CON_t^3)'))]

In [ ]:
# Inspect excluded countries that make log(STR) insignificant for log(CON_t^5).
results[(results['log(STR) p-value'] >= 0.05) & (results['Dependent variable'] == 'log(CON_t^5)')]

## 4.3 Time robustness.

In [ ]:
### PanelOLS: Fixed Effects with Log-Linear Time Trend. ###
results = pd.DataFrame()
for dep_var in dependent_vars:
    for i, exog_vars in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + exog_vars + ['t']).copy()
        temp_df = temp_df.set_index(['COUNTRY', 'YEAR'])
        y, X = temp_df[dep_var], temp_df[exog_vars+ ['t']]
        model = PanelOLS(y, X, entity_effects=True)
        res = model.fit(cov_type='clustered', cluster_entity=True)
        print(res.summary)

        # Extract result.
        row = {'Dependent variable': dep_var, 'Model ID': f'F{i}T', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared_within:.2f}"}
        for p in res.params.index:
            coef, se = res.params[p], res.std_errors[p]
            stars = ('***' if res.pvalues[p] < 0.001 else '**' if res.pvalues[p] < 0.01 else '*' if res.pvalues[p] < 0.05 else '')
            row[p] = f"{coef:.2f}{stars} ({se:.2f})"

        stars = ('***' if res.f_pooled.pval < 0.001 else '**' if res.f_pooled.pval < 0.01 else '*' if res.f_pooled.pval < 0.05 else '')
        row['F (poolability)'] = f'{res.f_pooled.stat:.2f}{stars}'

        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export results.
latex_table = generate_latex_table_with_results(results, params=['F (poolability)','log(STR)','log(POP)','log(SGDP)','log(POL)','t'])
latex_table = latex_table.replace('$R^{2}$', '$R^{2}$ (within)')
with open('tabs/fixed_time_trend.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results

In [ ]:
### PanelOLS: Fixed Effects with Time Fixed Effects. ###
results = pd.DataFrame()
for dep_var in dependent_vars:
    for i, exog_vars in enumerate(model_specs, start=1):

        # Run model.
        temp_df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[dep_var] + exog_vars).copy()
        temp_df = temp_df.set_index(['COUNTRY', 'YEAR'])
        model = PanelOLS(temp_df[dep_var], temp_df[exog_vars], entity_effects=True, time_effects=True)
        res = model.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
        print(res.summary)

        # Extract result.
        row = {'Dependent variable': dep_var, 'Model ID': f'F{i}TF', '$n$': f'{int(res.nobs)}', '$R^{2}$': f"{res.rsquared_within:.2f}"}
        for p in res.params.index:
            coef, se = res.params[p], res.std_errors[p]
            stars = ('***' if res.pvalues[p] < 0.001 else '**' if res.pvalues[p] < 0.01 else '*' if res.pvalues[p] < 0.05 else '')
            row[p] = f"{coef:.2f}{stars} ({se:.2f})"

        stars = ('***' if res.f_pooled.pval < 0.001 else '**' if res.f_pooled.pval < 0.01 else '*' if res.f_pooled.pval < 0.05 else '')
        row['F (poolability)'] = f'{res.f_pooled.stat:.2f}{stars}'
            
        results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

# Export results.
latex_table = generate_latex_table_with_results(results, params=['F (poolability)','log(STR)','log(POP)','log(SGDP)','log(POL)'])
latex_table = latex_table.replace('$R^{2}$', '$R^{2}$ (within)')
with open('tabs/time_fixed_effects.txt', 'w') as f:
    f.write(latex_table)

# Show results.
results